In [2]:
from google.colab import drive
drive.mount('/content/drive') # 구글 드라이브에 파일을 업로드하여 마운트하는 경우 이 셀 실행

Mounted at /content/drive


In [ ]:
# 코랩에서 드라이브 오류날 경우 이 셀로 테스트 해보세요!

# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)
wd.get("https://www.website.com")
print(wd.page_source)  # results

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/nae_house/url_sample_5184.csv')
df = df[3500:4200] # 본인이 맡은 부분으로 슬라이싱 변경

df['제목'] = 0 # 집들이 글 제목
df['작성일자'] = 0
df['평수'] = 0 # 평수 (1-9/10/20/30/40/50/60/70이상)
df['가족형태'] = 0 # 가족형태 (싱글라이프/신혼부부/아기가있는집/취학자녀가있는집/부모님과함께사는집/기타)
df['공간'] = 0 # 공간 (원룸&오피스텔/아파트/빌라&연합/단독주택/사무공간/상업공간/기타)
df['스타일'] = 0 # 스타일 (모던/미니멀&심플/내추럴/북유럽/빈티지&레트로/클래식&앤틱/프렌치&프로방스/러블리&로맨틱/인더스트리얼/한국&아시아/유니크&믹스매치)
df['세부공사'] = 0 # 세부공사 (헤링본 마루/대리석 바닥/ 원목마루/포세린타일/폴리싱타일/주방리모델링/조명시공/폴딩도어/중문/가벽&파티션/슬라이딩도어/아트월/발코니확장)
df['작업'] = 0 # 작업 (셀프,DIY/반셀프/전문가)
df['지역'] = 0 # 지역
df['분야'] = 0 # 분야 (리모델링/홈스타일링/부분공사/건축)
df['예산'] = 0 # 예산 (1백미만/1백/2백/3백/4백/5백/1천/2천/3천/4천이상)
df['스크랩'] = 0
df['좋아요'] = 0
df['댓글'] = 0
df['조회'] = 0

df

In [5]:
df.head() # 인덱스 번호 변경 확인

,url,제목,작성일자,평수,가족형태,공간,스타일,세부공사,작업,지역,분야,예산,스크랩,좋아요,댓글,조회
3500,https://ohou.se/projects/36604/detail?affect_t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3501,https://ohou.se/projects/36602/detail?affect_t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3502,https://ohou.se/projects/36595/detail?affect_t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3503,https://ohou.se/projects/36593/detail?affect_t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3504,https://ohou.se/projects/36583/detail?affect_t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
df.columns[2:]

Index(['작성일자', '평수', '가족형태', '공간', '스타일', '세부공사', '작업', '지역', '분야', '예산',
       '스크랩', '좋아요', '댓글', '조회'],
      dtype='object')

In [19]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as BS
import pandas as pd
from IPython.display import display
import warnings

In [ ]:
warnings.filterwarnings(action='ignore')

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome('chromedriver',options=options)
# 페이지가 모두 로드될 때까지 3초 기다려주도록 처리
driver.implicitly_wait(3)

for idx in df.index:
    # Selenium을 통해 실제 크롬창에서 접속한 것과 동일하게 해당 URL내용을 가져옴
    driver.get(df['url'][idx])
    # 소스를 읽어 BeautifulSoup를 이용해 파싱
    html = driver.page_source
    soup = BS(html, 'html.parser')


    reactions = soup.select('body > div.page > div > div > main > div.content-detail-footer > dl')
    if soup.find_all('h1', 'content-detail-header__title'):
        title = soup.select('body > div.page > div > div > main > div.content-detail-content-section > div.content-detail-content-section__content > header > h1')
        date = soup.select('body > div.page > div > div > main > div.content-detail-content-section > div.content-detail-content-section__content > header > div.content-detail-header__bottom > a > div.content-detail-header__author-date')
        texts = soup.select('body > div.page > div > div > main > div.content-detail-content-section > div.content-detail-content-section__content > section > div > div.project-detail-foldable__content > dl > div')
    else:
        contents = soup.select('body > div.page > div > div > main > div.content-detail-content-section.video-project-detail__content-section > div.content-detail-content-section__content')
        title = contents[0].find_all('h1', 'content-detail-header__title')
        date = soup.select('body > div.page > div > div > main > div.content-detail-content-section.video-project-detail__content-section > div.content-detail-content-section__content > div.css-v1ygcs.ebj05vv0 > header > div.content-detail-header__bottom > a > div.content-detail-header__author-date')
        texts = soup.select('body > div.page > div > div > main > div.content-detail-content-section.video-project-detail__content-section > div.content-detail-content-section__content > section > div.video-project-detail-upper-section__metadata-top > dl.project-detail-metadata-detail')[0].find_all('div')

    
    df['제목'][idx] = title[0].text
    df['작성일자'][idx] = date[0].text
    
    text_dic = {}
    for text in texts:
        text_dic[text.find('dt').text] = text.find('dd').text
    for react in reactions[0].find_all('div'):
        text_dic[react.find('dt').text] = react.find('dd').text

    for col in df.columns[2:]: 
        if col in text_dic:
            df[col][idx] = text_dic[col]

    if idx % 50 == 49:
        print(df['url'][idx])

driver.close()

In [ ]:
df.head()

In [ ]:
# 파일명 본인이 맡은 인덱스로 변경!
df.to_csv('3500-4200.csv')